Aprendizaje Automático · Grado en Ingeniería Informática · Curso 2022/23
---
Práctica 1: Predicción de la producción de energía solar
<br><br>
**Cuaderno 3 - Clasificación**
<br><br>
***Grupo Reducido 82 - Grupo de laboratorio 13***

# Importación de librerías y datos

Importamos las librerías necesarias

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pickle

Importamos los archivos necesarios

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/wind_ava.csv/wind_ava.csv')
predicciones = pd.read_csv('/content/drive/MyDrive/predicciones.csv')

columnas = df.columns[df.columns.str.endswith('13') | (df.columns == 'energy')]
df = df[columnas]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Comparación
Primero, dividiremos los datos de predicciones entre dos clases: alta y baja, utilizando el tercer quartil como referencia y comparar las predicciones.

In [ ]:
# Cargar el modelo
with open('/content/drive/MyDrive/modelo_final.pkl', 'rb') as f:
    modelo_final = pickle.load(f)

# Definir datos y etiquetas
x = df.drop(['energy'], axis=1)
y = df['energy']

# Dividimos las predicciones entre alta y baja utilizando el tercer quartil
q3 = df['energy'].quantile(0.75) # np.percentile(y, 75)

#baja = predicciones[df['energy']<=q3]
#alta = predicciones[df['energy']>q3]

baja = x[y <= q3]
alta = x[y > q3]

# Obtener las predicciones
predicciones_bajos = modelo_final.predict(baja)
predicciones_altos = modelo_final.predict(alta)

# comparar predicciones
# mae
#mae_baja = mean_absolute_error(y_true_baja, baja)
#mae_alta = mean_absolute_error(y_true_alta, alta)
mae_baja = mean_absolute_error(y[y <= q3], predicciones_bajos)
mae_alta = mean_absolute_error(y[y > q3], predicciones_altos)

# rmse
#rmse_baja = np.sqrt(mean_squared_error(y_true_baja, baja))
#rmse_alta = np.sqrt(mean_squared_error(y_true_alta, alta))
rmse_baja = np.sqrt(mean_squared_error(y[y <= q3], predicciones_bajos))
rmse_alta = np.sqrt(mean_squared_error(y[y > q3], predicciones_altos))

# Imprimir resultados
print("MAE para valores bajos de energía:", mae_baja)
print("RMSE para valores bajos de energía:", rmse_baja)
print("RMSE para valores altos de energía:", rmse_alta)
print("MAE para valores altos de energía:", mae_alta)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


MAE para valores bajos de energía: 362.5625058597615
RMSE para valores bajos de energía: 479.78450215987635
RMSE para valores altos de energía: 1733.1319643057097
MAE para valores altos de energía: 1686.3950061970172


Basándonos en los errores obtenidos, las predicciones son mejores para valores bajos de energía.

# Clasificación
Ahora, convertiremos el problema en uno de clasificación, con el tercer quartil como referencia y entrenaremos los datos con los diferentes modelos: knn, random forest, svm, decision trees, regresión logistica y redes neuronales.

Para ello, primero realizaremos una division de los datos en conjuntos de entrenamiento y prueba y evaluaremos el modelo tras su entrenamiento.

In [ ]:
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Convertimos y separamos los datos del problema

In [ ]:
# convertir el problema
df['clase'] = np.where(df['energy'] <= q3, 'baja', 'alta')

# separar x e y
X_clasificacion = df.drop(columns=['energy', 'clase'])
y_clasificacion = df['clase']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train_clasif, X_test_clasif, y_train_clasif, y_test_clasif = train_test_split(X_clasificacion, y_clasificacion, test_size=0.2, random_state=42)


Entrenamos los diferentes modelos y evaluamos su rendimiento

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# Crear y entrenar modelos
# Para KNN
modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_knn.fit(X_train_clasif, y_train_clasif)

# Para Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train_clasif, y_train_clasif)

# Para SVM
modelo_svm = SVC()
modelo_svm.fit(X_train_clasif, y_train_clasif)

# Para Decision Tree
modelo_dt = DecisionTreeClassifier(random_state=42)
modelo_dt.fit(X_train_clasif, y_train_clasif)

# Para Regresión Logística
modelo_lr = LogisticRegression(random_state=42)
modelo_lr.fit(X_train_clasif, y_train_clasif)

# Para Red Neuronal
modelo_mlp = MLPClassifier(random_state=42)
modelo_mlp.fit(X_train_clasif, y_train_clasif)

# Evaluar modelos
predicciones_clasif_knn = modelo_knn.predict(X_test_clasif)
predicciones_clasif_rf = modelo_rf.predict(X_test_clasif)
predicciones_clasif_svm = modelo_svm.predict(X_test_clasif)
predicciones_clasif_dt = modelo_dt.predict(X_test_clasif)
predicciones_clasif_lr = modelo_lr.predict(X_test_clasif)
predicciones_clasif_mlp = modelo_mlp.predict(X_test_clasif)

accuracy_knn = accuracy_score(y_test_clasif, predicciones_clasif_knn)
accuracy_rf = accuracy_score(y_test_clasif, predicciones_clasif_rf)
accuracy_svm = accuracy_score(y_test_clasif, predicciones_clasif_svm)
accuracy_dt = accuracy_score(y_test_clasif, predicciones_clasif_dt)
accuracy_lr = accuracy_score(y_test_clasif, predicciones_clasif_lr)
accuracy_mlp = accuracy_score(y_test_clasif, predicciones_clasif_mlp)

report_knn = classification_report(y_test_clasif, predicciones_clasif_knn)
report_rf = classification_report(y_test_clasif, predicciones_clasif_rf)
report_svm = classification_report(y_test_clasif, predicciones_clasif_svm)
report_dt = classification_report(y_test_clasif, predicciones_clasif_dt)
report_lr = classification_report(y_test_clasif, predicciones_clasif_lr)
report_mlp = classification_report(y_test_clasif, predicciones_clasif_mlp)

print("KNN - Accuracy:", accuracy_knn)
print("KNN - Classification Report:")
print(report_knn)

print("\nRandom Forest - Accuracy:", accuracy_rf)
print("Random Forest - Classification Report:")
print(report_rf)

print("\nSVM - Accuracy:", accuracy_svm)
print("SVM - Classification Report:")
print(report_svm)

print("\nDecision Tree - Accuracy:", accuracy_dt)
print("Decision Tree - Classification Report:")
print(report_dt)

print("\nRegresión Logística - Accuracy:", accuracy_lr)
print("Regresión Logística - Classification Report:")
print(report_lr)

print("\nRed Neuronal - Accuracy:", accuracy_mlp)
print("Red Neuronal - Classification Report:")
print(report_mlp)


KNN - Accuracy: 0.751578947368421
KNN - Classification Report:
              precision    recall  f1-score   support

        alta       0.49      0.37      0.42       232
        baja       0.81      0.88      0.84       718

    accuracy                           0.75       950
   macro avg       0.65      0.62      0.63       950
weighted avg       0.73      0.75      0.74       950


Random Forest - Accuracy: 0.8831578947368421
Random Forest - Classification Report:
              precision    recall  f1-score   support

        alta       0.80      0.70      0.74       232
        baja       0.91      0.94      0.92       718

    accuracy                           0.88       950
   macro avg       0.85      0.82      0.83       950
weighted avg       0.88      0.88      0.88       950


SVM - Accuracy: 0.78
SVM - Classification Report:
              precision    recall  f1-score   support

        alta       0.62      0.26      0.37       232
        baja       0.80      0.95     

El Random Forest tiene la mejor accuracy.

Vamos a aplicar grid search para ajustar los hiperparámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros a ajustar
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear el clasificador Random Forest
random_forest_classifier = RandomForestClassifier(random_state=42)

# Configurar Grid Search con validación cruzada
grid_search = GridSearchCV(estimator=random_forest_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ejecutar Grid Search en los datos de entrenamiento
grid_search.fit(X_train_clasif, y_train_clasif)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Mostrar la precisión del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_clasif, y_test_clasif)
print("Precisión del modelo Random Forest con los mejores hiperparámetros en el conjunto de prueba:", accuracy)


Mejores hiperparámetros encontrados:
{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Precisión del modelo Random Forest con los mejores hiperparámetros en el conjunto de prueba: 0.8810526315789474


El Grid Search no ha mejorado los hiperparametros, por lo que nos vamos a quedar con el modelo Random Forest con sus parámetros por omisión

# Predicciones


Sacamos las predicciones por clasificación y las guardamos en el csv predicciones_clasif, que tienen una accuracy del 88.32%

In [ ]:
# Crear el clasificador Random Forest con los mejores hiperparámetros
best_random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el clasificador Random Forest con los datos de entrenamiento
best_random_forest_classifier.fit(X_train_clasif, y_train_clasif)

# Realizar predicciones en el conjunto de prueba
predicciones = best_random_forest_classifier.predict(X_test_clasif)

# Crear un DataFrame con las predicciones y las etiquetas verdaderas
df_predicciones = pd.DataFrame({'Predicciones': predicciones, 'Etiqueta Verdadera': y_test_clasif})

# Guardar el DataFrame en un archivo CSV llamado 'predicciones_clasif.csv'
df_predicciones.to_csv('predicciones_clasif.csv', index=False)
print(df_predicciones)


     Predicciones Etiqueta Verdadera
1566         alta               alta
3159         baja               baja
538          baja               baja
2630         baja               baja
4420         baja               baja
...           ...                ...
4125         baja               baja
3400         baja               baja
1941         baja               baja
3679         baja               baja
2531         baja               baja

[950 rows x 2 columns]


# Conclusión


Tras convertir el problema inicial en uno de clasificación, hemos obtenido los siguientes resultado.

En primer lugar, tras evaluar diferentes modelos, hemos encontrado que Random Forest es aquel con el que obtenemos los mejores resultados, con una accuracy del 88,3%

A rasgos generales, observamos que las predicciones de valores altos son peores que aquellas para valores bajos, habiendo una diferencia notable en el rendimiento entre ambas predicciones, lo que puede producir cierta dificultad para hacer prediciones cuando la energía producida es alta.
